In [1]:
!pip install selenium
!pip install BeautifulSoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install webdriver_manager
!pip install pyperclip
!pip install fake-useragent

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
selenium 4.5.0 requires urllib3[socks]~=1.26, but you have urllib3 1.22 which is incompatible.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0, but you have google-api-core 2.11.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.16.0 which is incompatible.
cookiecutter 1.7.3 requires requests>=2.23.0, but you have requests 2.18.4 which is incompatible.
conda 22.9.0 requires requests>=2.20.1, but you have requests 2.18.4 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 1.22 which is incompatible.


  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached webdriver_manager-3.8.6-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np
import time
import requests
import pyperclip

In [92]:
def css_finds(css_selector):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def css_find(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def find(wait, css_selector):
    return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

def finds_xpath(xpath):
    return browser.find_elements(By.XPATH, xpath)

def find_xpath(xpath):
    return browser.find_element(By.XPATH, xpath)

def find_id(id_x):
    return browser.find_element(By.ID, id_x)

# Options Setting
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')
# options.add_argument('headless')
# Header Setting
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)


# Settings
NAVER_ID = "sjk5838"
NAVER_PW = "rlatjdwls00@K"

CAFENAME = "1motion1"
BORADTITLE = "전체글보기"

keyword = "3D"
COMMENTS = "테스트 댓글 입니다."

# def 1
# Crawling Start
browser.get("https://nid.naver.com/nidlogin.login")
browser.implicitly_wait(2)

input_id = find_id('id')
input_pw = find_id('pw')

time.sleep(2)

pyperclip.copy(NAVER_ID) 
input_id.send_keys(Keys.CONTROL, "v")

pyperclip.copy(NAVER_PW) 
input_pw.send_keys(Keys.CONTROL, "v")
input_pw.send_keys("\n")

# Not needed when it's headless
# no_save_btn = find_id('new.dontsave')
# no_save_btn.click()

time.sleep(1)

# def 2
browser.get(f"https://cafe.naver.com/{CAFENAME}")
time.sleep(2)

boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')

boardName.click()

time.sleep(2)

browser.switch_to.frame("cafe_main")

browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

time.sleep(1)

soup = BS(browser.page_source, "html.parser")
soup = soup.find_all(class_='article-board m-tcol-c')[1]

datas = soup.find_all(class_='td_article')
dates = soup.find_all(class_='td_date')

a_hrefs = soup.find_all("a")

# def 3
post_hrefs = []
for href in a_hrefs:
    if keyword in href.text:
        post_hrefs.append(href["href"])

final_hrefs = []

for href in post_hrefs:
    parsed_url = urlparse(href)
    query_params = parse_qs(parsed_url.query)
    article_id = query_params['articleid'][0]
    club_id = query_params['clubid'][0]
    new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"
    
    final_hrefs.append(new_url)

# def 4
for r_href in final_hrefs:
    browser.get(r_href)
    browser.switch_to.frame("cafe_main")
    
    text_area = find_xpath('//*[@id="app"]/div/div/div[2]/div[2]/div[5]/div[2]/div[1]/textarea')
    text_area.click()

pyperclip.copy(COMMENTS) 
text_area.send_keys(Keys.CONTROL, "v")

time.sleep(1)

browser.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div[2]/div[5]/div[2]/div[2]/div[2]/a').click()

browser.close()



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app"]/div/div/div[2]/div[2]/div[5]/div[2]/div[1]/textarea"}
  (Session info: chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0031A813+48355]
	(No symbol) [0x002AC4B1]
	(No symbol) [0x001B5358]
	(No symbol) [0x001E09A5]
	(No symbol) [0x001E0B3B]
	(No symbol) [0x0020E232]
	(No symbol) [0x001FA784]
	(No symbol) [0x0020C922]
	(No symbol) [0x001FA536]
	(No symbol) [0x001D82DC]
	(No symbol) [0x001D93DD]
	GetHandleVerifier [0x0057AABD+2539405]
	GetHandleVerifier [0x005BA78F+2800735]
	GetHandleVerifier [0x005B456C+2775612]
	GetHandleVerifier [0x003A51E0+616112]
	(No symbol) [0x002B5F8C]
	(No symbol) [0x002B2328]
	(No symbol) [0x002B240B]
	(No symbol) [0x002A4FF7]
	BaseThreadInitThunk [0x764C7D59+25]
	RtlInitializeExce

In [84]:
from urllib.parse import urlparse, parse_qs

final_hrefs = []

for href in post_hrefs:
    parsed_url = urlparse(href)
    query_params = parse_qs(parsed_url.query)
    article_id = query_params['articleid'][0]
    club_id = query_params['clubid'][0]
    new_url = f"https://cafe.naver.com/1motion1?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"
    
    final_hrefs.append(new_url)

In [96]:
for po in post_hrefs:
    da_1 = po.split("&")
    print(da_1)

In [ ]:
comments = browser.find_elements(By.CSS_SELECTOR, '.comment')
for comment in comments:
    nickname = comment.find_element_by_css_selector('.user_name').text
    
    if nickname == my_nickname:
        # 다른 URL로 넘어가는 로직 작성
        continue
    
    comment_textarea = comment.find_element_by_css_selector('.comment_input textarea')
    comment_textarea.send_keys(comment_content)
    comment_button = comment.find_element_by_css_selector('.comment_input .comment_submit')
    comment_button.click()

browser.quit()

import math

crawling_page = int(math.ceil(100 / 50)+1)
print(crawling_page)

---------------------------

# Naver Cafe Macro Serach Code

In [531]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np
import time
import requests
import pyperclip

def css_finds(css_selector):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def css_find(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def finds_xpath(xpath):
    return browser.find_elements(By.XPATH, xpath)

def find_xpath(xpath):
    return browser.find_element(By.XPATH, xpath)

def find_id(id_x):
    return browser.find_element(By.ID, id_x)

def find_classname(class_name):
    return browser.find_element(By.CLASS_NAME, class_name)

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--start-maximized')
options.add_argument("--window-size=1080,800")
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)   

In [532]:
# Settings
NAVER_ID = "sjk5838"
NAVER_PW = "rlatjdwls00@K"

CAFENAME = "1motion1"
BORADTITLE = "김성진"
NICKNAME = "최고의일베충소정"

keyword = "3D"
COMMENTS = "Test Comments"

# def 1
# Crawling Start

browser.get("https://nid.naver.com/nidlogin.login")
browser.implicitly_wait(2)

input_id = find_id('id')
input_pw = find_id('pw')

time.sleep(2)

pyperclip.copy(NAVER_ID)
input_id.send_keys(Keys.CONTROL, "v")

pyperclip.copy(NAVER_PW) 
input_pw.send_keys(Keys.CONTROL, "v")
input_pw.send_keys("\n")

# Not needed when it's headless
no_save_btn = find_id('new.dontsave')
no_save_btn.click()

time.sleep(1)

In [533]:
# def 2
browser.get(f"https://cafe.naver.com/{CAFENAME}")
time.sleep(3)

In [496]:
search_input = find_id('topLayerQueryInput')

pyperclip.copy(keyword)
search_input.send_keys(Keys.CONTROL, 'v')
search_input.send_keys('\n')

In [497]:
browser.switch_to.frame("cafe_main")
time.sleep(2)

In [498]:
find_xpath('//*[@id="currentSearchByTop"]').click()
time.sleep(1)
find_xpath('//*[@id="sl_general"]/li[2]/a').click()
find_xpath('//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button').click()

In [499]:
find_xpath('//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button').click()

In [500]:
find_xpath('//*[@id="listSizeSelectDiv"]/a').click()
find_xpath('//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

In [502]:
page_numbers_btn = css_finds('div.prev-next > a')

In [503]:
page_nums = []
for i in page_numbers_btn:
    page_nums.append(i.text)

In [506]:
final_hrefs = []

if len(page_nums) > 1:
    for i in page_nums[:-1]:
        if i != 1: 
            browser.find_element(By.LINK_TEXT, f"{i}").click()
            
        soup = BS(browser.page_source, "html.parser")
        soup = soup.find_all(class_='article-board result-board m-tcol-c')[0]

        a_hrefs = soup.find_all("a")

        # def 3
        post_hrefs = []
        for href in a_hrefs:
            if keyword in href.text:
                post_hrefs.append(href["href"])

        for href in post_hrefs:
            if (href == '#'):
                pass
            else:
                parsed_url = urlparse(href)
                query_params = parse_qs(parsed_url.query)
                article_id = query_params['articleid'][0]
                club_id = query_params['clubid'][0]
                new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

                final_hrefs.append(new_url)
        
else:
    soup = BS(browser.page_source, "html.parser")
    soup = soup.find_all(class_='article-board result-board m-tcol-c')

    a_hrefs = soup.find_all("a")

    # def 3
    post_hrefs = []
    for href in a_hrefs:
        if keyword in href.text:
            post_hrefs.append(href["href"])

    for href in post_hrefs:
        if (href == '#'):
            pass
        else:
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            article_id = query_params['articleid'][0]
            club_id = query_params['clubid'][0]
            new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

            final_hrefs.append(new_url)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [517]:
# cmtnicks = []

# for p_href in final_hrefs:
#     browser.get(p_href)
#     time.sleep(1)
#     browser.switch_to.frame("cafe_main")
#     time.sleep(1)
    
#     try:
# #         nicksname = browser.find_element(By.CLASS_NAME, 'comment_inbox_name').text
#         nickname = NICKNAME
#         cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')

#         if cmtNicks:
#             for cmtNick in cmtNicks:
#                 cmtnick = cmtNick.text
#                 cmtnicks.append(cmtnick)

#             if nickname in cmtnicks:
#                 # 내 닉네임으로 댓글이 있는 경우, 다음 페이지로 넘어감
#                 continue
#             else:
#                 # 내 닉네임으로 댓글이 없는 경우, 댓글 작성
#                 time.sleep(1)
#                 text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
#                 text_area.click()

#                 pyperclip.copy(COMMENTS) 
#                 text_area.send_keys(Keys.CONTROL, "v")
#                 register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
#                 register_btn.click()
#         else:
#             # 댓글 작성
#             time.sleep(1)
#             text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
#             text_area.click()

#             pyperclip.copy(COMMENTS) 
#             text_area.send_keys(Keys.CONTROL, "v")
#             register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
#             register_btn.click()
            
#     except NoSuchElementException:
#         pass

#     cmtnicks.clear()

# Nave Cafe Macro Start Code

In [534]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np
import time
import requests
import pyperclip

def css_finds(css_selector):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def css_find(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def finds_xpath(xpath):
    return browser.find_elements(By.XPATH, xpath)

def find_xpath(xpath):
    return browser.find_element(By.XPATH, xpath)

def find_id(id_x):
    return browser.find_element(By.ID, id_x)

def find_classname(class_name):
    return browser.find_element(By.CLASS_NAME, class_name)

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--start-maximized')
options.add_argument("--window-size=1080,800")
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)   

### def 1

In [549]:
# Settings
NAVER_ID = "sjk5838"
NAVER_PW = "rlatjdwls00@K"

CAFENAME = "1motion1"
BORADTITLE = "김성진"
NICKNAME = "최고의일베충소정"

keyword = "3D"
COMMENTS = "Test Comments"

# def 1
# Crawling Start

browser.get("https://nid.naver.com/nidlogin.login")
browser.implicitly_wait(2)

input_id = find_id('id')
input_pw = find_id('pw')

time.sleep(2)

pyperclip.copy(NAVER_ID)
input_id.send_keys(Keys.CONTROL, "v")

pyperclip.copy(NAVER_PW) 
input_pw.send_keys(Keys.CONTROL, "v")
input_pw.send_keys("\n")

# Not needed when it's headless
no_save_btn = find_id('new.dontsave')
no_save_btn.click()

time.sleep(1)

### def 2

In [550]:
# def 2
browser.get(f"https://cafe.naver.com/{CAFENAME}")
time.sleep(3)

In [551]:
search_input = find_id('topLayerQueryInput')

pyperclip.copy(keyword)
search_input.send_keys(Keys.CONTROL, 'v')
search_input.send_keys('\n')

In [552]:
browser.switch_to.frame("cafe_main")
time.sleep(2)

In [553]:
find_xpath('//*[@id="currentSearchByTop"]').click()
time.sleep(1)
find_xpath('//*[@id="sl_general"]/li[2]/a').click()
find_xpath('//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button').click()

In [555]:
find_xpath('//*[@id="listSizeSelectDiv"]/a').click()
find_xpath('//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

In [556]:
page_numbers_btn = css_finds('div.prev-next > a')

In [557]:
page_nums = []
for i in page_numbers_btn:
    page_nums.append(i.text)

In [558]:
final_hrefs = []

if len(page_nums) > 1:
    for i in page_nums[:-1]:
        if i != 1: 
            browser.find_element(By.LINK_TEXT, f"{i}").click()
            
        soup = BS(browser.page_source, "html.parser")
        soup = soup.find_all(class_='article-board m-tcol-c')[1]

        a_hrefs = soup.find_all("a")

        # def 3
        post_hrefs = []
        for href in a_hrefs:
            if keyword in href.text:
                post_hrefs.append(href["href"])

        for href in post_hrefs:
            if (href == '#'):
                pass
            else:
                parsed_url = urlparse(href)
                query_params = parse_qs(parsed_url.query)
                article_id = query_params['articleid'][0]
                club_id = query_params['clubid'][0]
                new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

                final_hrefs.append(new_url)
        
else:
    soup = BS(browser.page_source, "html.parser")
    soup = soup.find_all(class_='article-board m-tcol-c')[1]

    a_hrefs = soup.find_all("a")

    # def 3
    post_hrefs = []
    for href in a_hrefs:
        if keyword in href.text:
            post_hrefs.append(href["href"])

    for href in post_hrefs:
        if (href == '#'):
            pass
        else:
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            article_id = query_params['articleid'][0]
            club_id = query_params['clubid'][0]
            new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

            final_hrefs.append(new_url)

In [559]:
len(final_hrefs)

14

### def 3

In [560]:
cmtnicks = []

for p_href in final_hrefs:
    browser.get(p_href)
    time.sleep(1)
    browser.switch_to.frame("cafe_main")
    time.sleep(1)
    
    try:
#         nicksname = browser.find_element(By.CLASS_NAME, 'comment_inbox_name').text
        nickname = NICKNAME
        cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')

        if cmtNicks:
            for cmtNick in cmtNicks:
                cmtnick = cmtNick.text
                cmtnicks.append(cmtnick)

            if nickname in cmtnicks:
                # 내 닉네임으로 댓글이 있는 경우, 다음 페이지로 넘어감
                continue
            else:
                # 내 닉네임으로 댓글이 없는 경우, 댓글 작성
                time.sleep(1)
                text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                text_area.click()

                pyperclip.copy(COMMENTS) 
                text_area.send_keys(Keys.CONTROL, "v")
                register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                register_btn.click()
        else:
            # 댓글 작성
            time.sleep(1)
            text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
            text_area.click()

            pyperclip.copy(COMMENTS) 
            text_area.send_keys(Keys.CONTROL, "v")
            register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
            register_btn.click()
            
    except NoSuchElementException:
        pass

    cmtnicks.clear()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0041A813+48355]
	(No symbol) [0x003AC4B1]
	(No symbol) [0x002B5358]
	(No symbol) [0x0029D293]
	(No symbol) [0x002FE37B]
	(No symbol) [0x0030C473]
	(No symbol) [0x002FA536]
	(No symbol) [0x002D82DC]
	(No symbol) [0x002D93DD]
	GetHandleVerifier [0x0067AABD+2539405]
	GetHandleVerifier [0x006BA78F+2800735]
	GetHandleVerifier [0x006B456C+2775612]
	GetHandleVerifier [0x004A51E0+616112]
	(No symbol) [0x003B5F8C]
	(No symbol) [0x003B2328]
	(No symbol) [0x003B240B]
	(No symbol) [0x003A4FF7]
	BaseThreadInitThunk [0x76767D59+25]
	RtlInitializeExceptionChain [0x77E9B74B+107]
	RtlClearBits [0x77E9B6CF+191]


-----------------------------------------

--------------

In [28]:
cmtnicks = []

for p_href in final_hrefs:
    browser.get(p_href)
    time.sleep(1)
    browser.switch_to.frame("cafe_main")
    time.sleep(1)
    
    nickname = NICKNAME

    cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')
    
    if cmtNicks:
        for cmtNick in cmtNicks:
            cmtnick = cmtNick.text
            cmtnicks.append(cmtnick)

        if nickname in cmtnicks:
            # 내 닉네임으로 댓글이 있는 경우, 다음 페이지로 넘어감
            continue
        else:
            # 내 닉네임으로 댓글이 없는 경우, 댓글 작성
            time.sleep(1)
            text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
            text_area.click()

            pyperclip.copy(COMMENTS) 
            text_area.send_keys(Keys.CONTROL, "v")
            browser.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div[2]/div[5]/div[2]/div[2]/div[2]/a').click()
    else:
        # 댓글 작성
        time.sleep(1)
        text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
        text_area.click()

        pyperclip.copy(COMMENTS) 
        text_area.send_keys(Keys.CONTROL, "v")
        browser.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div[2]/div[5]/div[2]/div[2]/div[2]/a').click()

    cmtnicks.clear()

In [61]:
def crawling_start(NAVER_ID, NAVER_PW, CAFENAME, keyword, COMMENTS):
    def selenuim_setting():
        # Options Setting
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('no-sandox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--start-maximized')
        options.add_argument('incognito')
        # options.add_argument('headless')
        # Header Setting
        service = Service(ChromeDriverManager().install())
        browser = webdriver.Chrome(service=service, options=options)

        return browser

    def naverMarco(browser):
        def css_finds(css_selector):
            return browser.find_elements(By.CSS_SELECTOR, css_selector)

        def css_find(css_selector):
            return browser.find_element(By.CSS_SELECTOR, css_selector)

        def find(wait, css_selector):
            return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

        def finds_xpath(xpath):
            return browser.find_elements(By.XPATH, xpath)

        def find_xpath(xpath):
            return browser.find_element(By.XPATH, xpath)

        def find_id(id_x):
            return browser.find_element(By.ID, id_x)

        
        browser.get("https://nid.naver.com/nidlogin.login")
        browser.implicitly_wait(2)

        input_id = find_id('id')
        input_pw = find_id('pw')

        time.sleep(2)

        pyperclip.copy(NAVER_ID) 
        input_id.send_keys(Keys.CONTROL, "v")

        pyperclip.copy(NAVER_PW) 
        input_pw.send_keys(Keys.CONTROL, "v")
        input_pw.send_keys("\n")

        time.sleep(1)
        
        browser.get(f"https://cafe.naver.com/{CAFENAME}")
        
        time.sleep(2)

        boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')
        boardName.click()

        time.sleep(2)

        browser.switch_to.frame("cafe_main")

        browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
        browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

        time.sleep(1)

        soup = BS(browser.page_source, "html.parser")
        soup = soup.find_all(class_='article-board m-tcol-c')[1]

        datas = soup.find_all(class_='td_article')
        dates = soup.find_all(class_='td_date')

        a_hrefs = soup.find_all("a")
        
        
        post_hrefs = []
        
        for href in a_hrefs:
            if keyword in href.text:
                post_hrefs.append(href["href"])

                
        final_hrefs = []

        for href in post_hrefs:
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            article_id = query_params['articleid'][0]
            club_id = query_params['clubid'][0]
            new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

            final_hrefs.append(new_url)

            
        cmtnicks = []

        for p_href in final_hrefs:
            browser.get(p_href)
            time.sleep(1)
            browser.switch_to.frame("cafe_main")
            time.sleep(1)

            try:
                nicksname = browser.find_element(By.CLASS_NAME, 'comment_inbox_name').text
                cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')

                if cmtNicks:
                    for cmtNick in cmtNicks:
                        cmtnick = cmtNick.text
                        cmtnicks.append(cmtnick)

                    if nickname in cmtnicks:
                        # 내 닉네임으로 댓글이 있는 경우, 다음 페이지로 넘어감
                        continue
                    else:
                        # 내 닉네임으로 댓글이 없는 경우, 댓글 작성
                        time.sleep(1)
                        text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                        text_area.click()

                        pyperclip.copy(COMMENTS) 
                        text_area.send_keys(Keys.CONTROL, "v")
                        register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                        register_btn.click()
                else:
                    # 댓글 작성
                    time.sleep(1)
                    text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                    text_area.click()

                    pyperclip.copy(COMMENTS) 
                    text_area.send_keys(Keys.CONTROL, "v")
                    register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                    register_btn.click()

            except NoSuchElementException:
                pass

            cmtnicks.clear()
            
            df = pd.DataFrame(final_hrefs)
            
            return df

SyntaxError: non-default argument follows default argument (236959528.py, line 1)

In [140]:
for i in cmtNicks:
    print(i.text)

In [116]:
print(final_hrefs[1])

https://cafe.naver.com/1motion1?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D28520242%2526page%3D1%2526boardtype%3DL%2526articleid%3D559%2526referrerAllArticles%3Dtrue


In [1]:
import pandas as pd

In [100]:
df = pd.read_csv("C://Users//user//Downloads//test_travel.csv")

In [101]:
df.isnull().sum().sum()

2

In [102]:
df = df.fillna(0)

In [117]:
df2 = df[df['연락처'].str.contains("\+82" , na=False)]

In [119]:
df2['이름']

21    박영준 (Brad Park)
67                이호철
71                정소정
Name: 이름, dtype: object

In [116]:
for i in df2['연락처']:
    tt = i.split("+82")[1]
    t2 = tt.split("휴대폰")[0]
    t3 = '0'+t2[1:]
    

010-8262-6228
010-9612-2352
010-3654-1504


In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np
import time
import requests
import pyperclip


def naverCafeMarco(NAVER_ID, NAVER_PW, CAFENAME, BORADTITLE, NICKNAME, keyword, COMMENTS):

    def css_finds(css_selector):
        return browser.find_elements(By.CSS_SELECTOR, css_selector)

    def css_find(css_selector):
        return browser.find_element(By.CSS_SELECTOR, css_selector)

    def find(wait, css_selector):
        return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

    def finds_xpath(xpath):
        return browser.find_elements(By.XPATH, xpath)

    def find_xpath(xpath):
        return browser.find_element(By.XPATH, xpath)

    def find_id(id_x):
        return browser.find_element(By.ID, id_x)

    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('no-sandox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--start-maximized')
    options.add_argument('incognito')

    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service, options=options)    

    # def 1
    # Crawling Start

    browser.get("https://nid.naver.com/nidlogin.login")
    browser.implicitly_wait(2)

    input_id = find_id('id')
    input_pw = find_id('pw')

    time.sleep(2)

    pyperclip.copy(NAVER_ID)
    input_id.send_keys(Keys.CONTROL, "v")

    pyperclip.copy(NAVER_PW) 
    input_pw.send_keys(Keys.CONTROL, "v")
    input_pw.send_keys("\n")

    # Not needed when it's headless
    # no_save_btn = find_id('new.dontsave')
    # no_save_btn.click()

    time.sleep(1)

    # def 2
    browser.get(f"https://cafe.naver.com/{CAFENAME}")
    time.sleep(2)

    boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')

    boardName.click()

    time.sleep(2)

    browser.switch_to.frame("cafe_main")

    browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
    browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

    time.sleep(1)

    soup = BS(browser.page_source, "html.parser")
    soup = soup.find_all(class_='article-board m-tcol-c')[1]

    datas = soup.find_all(class_='td_article')
    dates = soup.find_all(class_='td_date')

    a_hrefs = soup.find_all("a")

    # def 3
    post_hrefs = []
    for href in a_hrefs:
        if keyword in href.text:
            post_hrefs.append(href["href"])

    final_hrefs = []

    for href in post_hrefs:
        parsed_url = urlparse(href)
        query_params = parse_qs(parsed_url.query)
        article_id = query_params['articleid'][0]
        club_id = query_params['clubid'][0]
        new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

        final_hrefs.append(new_url)

    while len(final_hrefs) == 0:
        # final_hrefs가 빈 리스트인 경우, 계속해서 반복적으로 돌립니다.
        time.sleep(1)
        browser.refresh()

        boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')

        boardName.click()

        time.sleep(2)

        browser.switch_to.frame("cafe_main")

        browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
        browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

        time.sleep(1)

        soup = BS(browser.page_source, "html.parser")
        soup = soup.find_all(class_='article-board m-tcol-c')[1]

        datas = soup.find_all(class_='td_article')
        dates = soup.find_all(class_='td_date')

        a_hrefs = soup.find_all("a")

        # def 3
        post_hrefs = []
        for href in a_hrefs:
            if keyword in href.text:
                post_hrefs.append(href["href"])

        final_hrefs = []

        for href in post_hrefs:
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            article_id = query_params['articleid'][0]
            club_id = query_params['clubid'][0]
            new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

            final_hrefs.append(new_url)
            print("해당 키워드를 가진 게시물을 찾았습니다.")


    cmtnicks = []

    for p_href in final_hrefs:
        browser.get(p_href)
        time.sleep(1)
        browser.switch_to.frame("cafe_main")
        time.sleep(1)

        try:
    #         nicksname = browser.find_element(By.CLASS_NAME, 'comment_inbox_name').text
            nickname = NICKNAME
            cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')

            if cmtNicks:
                for cmtNick in cmtNicks:
                    cmtnick = cmtNick.text
                    cmtnicks.append(cmtnick)

                if nickname in cmtnicks:
                    # 내 닉네임으로 댓글이 있는 경우, 다음 페이지로 넘어감
                    print("해당 게시물은 이미 댓글을 작성한 적이 있는 게시물 이므로 다음 페이지로 넘어갑니다.")
                    continue
                else:
                    # 내 닉네임으로 댓글이 없는 경우, 댓글 작성
                    time.sleep(1)
                    text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                    text_area.click()

                    pyperclip.copy(COMMENTS)
                    text_area.send_keys(Keys.CONTROL, "v")
                    register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                    register_btn.click()
                    print("새로운 댓글을 작성 하였습니다.")
            else:
                # 댓글 작성
                time.sleep(1)
                text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                text_area.click()

                pyperclip.copy(COMMENTS) 
                text_area.send_keys(Keys.CONTROL, "v")
                register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                register_btn.click()
                print("새로운 댓글을 작성 하였습니다.")

        except NoSuchElementException:
            pass

        cmtnicks.clear()
        print("매크로를 종료합니다.")
        time.sleep(3)
        
    browser.close()


In [566]:
NAVER_ID = "sjk5838"
NAVER_PW = "rlatjdwls00@K"

CAFENAME = "1motion1"
BORADTITLE = "전체글보기"
NICKNAME = "최고의일베충소정"

keyword = "3D"
COMMENTS = "Test Comments"

In [32]:
naverCafeMarco(NAVER_ID, NAVER_PW, CAFENAME, BORADTITLE, NICKNAME, keyword, COMMENTS)

해당 게시물은 이미 댓글을 작성한 적이 있는 게시물 이므로 다음 페이지로 넘어갑니다.
해당 게시물은 이미 댓글을 작성한 적이 있는 게시물 이므로 다음 페이지로 넘어갑니다.


In [19]:
df = pd.DataFrame(final_hrefs)
df

,0
0,https://cafe.naver.com/1motion1?iframe_url_utf...
1,https://cafe.naver.com/1motion1?iframe_url_utf...


In [398]:
df = pd.read_csv("C://Users//user//Downloads//음식_2023-06-26_145817_utf-8.csv")

In [407]:
df = pd.read_csv("C://Users//user//Downloads//음식_2023-06-26_145817_utf-8.csv")

df = df.fillna('null')

numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
        
df.isnull().sum()
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
        
df['출생연도'] = int_values

df.columns = ['번호', '이름', '연락처', '지역', '출생연도']

df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')

if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
    
df.index[df['연락처'].str.contains('1')]
    
phone_num = df.index[df['연락처'].str.contains('1')]
df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']

In [390]:
df = df.fillna('null')

In [399]:
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')

In [400]:
df.isnull().sum()
df = df.fillna('null')
numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
int_values = []
for item in numbers:
    if item.isdigit():
        int_values.append(int(item))
    else:
        int_values.append('없음')
df['출생연도'] = int_values
df.columns = ['번호', '이름', '연락처', '지역', '출생연도']

In [401]:
df['연락처'].isnull().sum()

0

In [402]:
df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')

In [403]:
df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')

if len(df[df['연락처'].str.contains('\+82')]) >= 1:
    df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
    
df.index[df['연락처'].str.contains('1')]
    
phone_num = df.index[df['연락처'].str.contains('1')]
df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']

Int64Index([2], dtype='int64')

In [404]:
phone_num = df.index[df['연락처'].str.contains('1')]
df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']

In [405]:
df.loc[phone_num, '연락처']

2    011-638-1205휴
Name: 연락처, dtype: object

In [296]:
int_numbers = []

for i in df['연락처']:
    if "10-" in i:
        ii = "0" + i[1:]
        int_numbers.append(ii)
    else:
        int_numbers.append('없음')
df['연락처'] = int_numbers
#         df.to_csv(f"{f_name}_cp949.csv", encoding='CP949')

---------

------------

In [562]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from urllib.parse import urlparse, parse_qs

import pandas as pd
import numpy as np
import time
import requests
import pyperclip

def css_finds(css_selector):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def css_find(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

def find(wait, css_selector):
    return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

def finds_xpath(xpath):
    return browser.find_elements(By.XPATH, xpath)

def find_xpath(xpath):
    return browser.find_element(By.XPATH, xpath)

def find_id(id_x):
    return browser.find_element(By.ID, id_x)

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)    

In [567]:
# def 1
# Crawling Start

browser.get("https://nid.naver.com/nidlogin.login")
browser.implicitly_wait(2)

input_id = find_id('id')
input_pw = find_id('pw')

time.sleep(2)

pyperclip.copy(NAVER_ID)
input_id.send_keys(Keys.CONTROL, "v")

pyperclip.copy(NAVER_PW) 
input_pw.send_keys(Keys.CONTROL, "v")
input_pw.send_keys("\n")

# Not needed when it's headless
no_save_btn = find_id('new.dontsave')
no_save_btn.click()

time.sleep(1)

In [568]:
# def 2
browser.get(f"https://cafe.naver.com/{CAFENAME}")
time.sleep(2)

boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')

boardName.click()

time.sleep(2)

browser.switch_to.frame("cafe_main")

In [569]:
browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

time.sleep(1)

In [571]:
page_numbers_btn = css_finds('div.prev-next > a')

page_nums = []
for i in page_numbers_btn:
    page_nums.append(i.text)

In [574]:
final_hrefs = []

if len(page_nums) > 1:
    for i in range(1, 4):
        browser.find_element(By.LINK_TEXT, f"{i}").click()
        
        soup = BS(browser.page_source, "html.parser")
        soup = soup.find_all(class_='article-board m-tcol-c')[1]

        datas = soup.find_all(class_='td_article')
        dates = soup.find_all(class_='td_date')

        a_hrefs = soup.find_all("a")
        
        # def 3
        post_hrefs = []
        for href in a_hrefs:
            if keyword in href.text:
                post_hrefs.append(href["href"])

        for href in post_hrefs:
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            article_id = query_params['articleid'][0]
            club_id = query_params['clubid'][0]
            new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

            final_hrefs.append(new_url)

1
2
3


In [561]:







soup = BS(browser.page_source, "html.parser")
soup = soup.find_all(class_='article-board m-tcol-c')[1]

datas = soup.find_all(class_='td_article')
dates = soup.find_all(class_='td_date')

a_hrefs = soup.find_all("a")

# def 3
post_hrefs = []
for href in a_hrefs:
    if keyword in href.text:
        post_hrefs.append(href["href"])

final_hrefs = []

for href in post_hrefs:
    parsed_url = urlparse(href)
    query_params = parse_qs(parsed_url.query)
    article_id = query_params['articleid'][0]
    club_id = query_params['clubid'][0]
    new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

    final_hrefs.append(new_url)

while len(final_hrefs) == 0:
    # If final_hrefs is an empty list, continue to rotate repeatedly.
    time.sleep(1)
    browser.refresh()

    boardName = browser.find_element(By.LINK_TEXT, f'{BORADTITLE}')

    boardName.click()

    time.sleep(2)

    browser.switch_to.frame("cafe_main")

    browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
    browser.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

    time.sleep(1)

    soup = BS(browser.page_source, "html.parser")
    soup = soup.find_all(class_='article-board m-tcol-c')[1]

    datas = soup.find_all(class_='td_article')
    dates = soup.find_all(class_='td_date')

    a_hrefs = soup.find_all("a")

    # def 3
    post_hrefs = []
    for href in a_hrefs:
        if keyword in href.text:
            post_hrefs.append(href["href"])

    final_hrefs = []

    for href in post_hrefs:
        parsed_url = urlparse(href)
        query_params = parse_qs(parsed_url.query)
        article_id = query_params['articleid'][0]
        club_id = query_params['clubid'][0]
        new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

        final_hrefs.append(new_url)

    soup = BS(browser.page_source, "html.parser")
    soup = soup.find_all(class_='article-board m-tcol-c')[1]

    datas = soup.find_all(class_='td_article')
    dates = soup.find_all(class_='td_date')

    a_hrefs = soup.find_all("a")

    post_hrefs = []
    for href in a_hrefs:
        if keyword in href.text:
            post_hrefs.append(href["href"])

    final_hrefs = []

    for href in post_hrefs:
        parsed_url = urlparse(href)
        query_params = parse_qs(parsed_url.query)
        article_id = query_params['articleid'][0]
        club_id = query_params['clubid'][0]
        new_url = f"https://cafe.naver.com/{CAFENAME}?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D{club_id}%2526page%3D1%2526boardtype%3DL%2526articleid%3D{article_id}%2526referrerAllArticles%3Dtrue"

        final_hrefs.append(new_url)


cmtnicks = []

for p_href in final_hrefs:
    browser.get(p_href)
    time.sleep(1)
    browser.switch_to.frame("cafe_main")
    time.sleep(1)

    try:
#         nicksname = browser.find_element(By.CLASS_NAME, 'comment_inbox_name').text
        nickname = NICKNAME
        cmtNicks = browser.find_elements(By.CLASS_NAME, 'comment_nickname')

        if cmtNicks:
            for cmtNick in cmtNicks:
                cmtnick = cmtNick.text
                cmtnicks.append(cmtnick)

            if nickname in cmtnicks:
                continue
            else:
                time.sleep(1)
                text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
                text_area.click()

                pyperclip.copy(COMMENTS) 
                text_area.send_keys(Keys.CONTROL, "v")
                register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
                register_btn.click()
        else:
            # Write Comment
            time.sleep(1)
            text_area = browser.find_element(By.CLASS_NAME, 'comment_inbox_text')
            text_area.click()

            pyperclip.copy(COMMENTS) 
            text_area.send_keys(Keys.CONTROL, "v")
            register_btn = browser.find_element(By.CLASS_NAME, 'btn_register')
            register_btn.click()

    except NoSuchElementException:
        pass

    cmtnicks.clear()

time.sleep(3)
browser.close()

title = []

for da in datas:
    art_title = da.find(class_='article')
    art_title.get_text().strip()
    title.append(art_title)
